## Dummy workflow

In [27]:
#!pip uninstall gensim -y
#!pip install gensim==3.8.1
#!pip uninstall scikit-learn -y
#!pip install scikit_learn==0.19.1
#!python --version
#!pip install pandas
#!pip install statsmodels
#!pip install flask

# change the ENV to python36!!!!!

### To do's:
- API-Engines Response
- Input Format anpassen
- eine Klasse über den Prozess schreiben?
- Zertifikat auf api.commonsign.com testen, ob es ein Server Zertifikat ist, QWAC Format

In [5]:
from utils import *

In [6]:
# Map data
#data = pd.read_csv("CommonSign_Demo_Dataset.csv",sep=",")
file_name = "CommonSign_DEV.csv"
dataNode = commonsignEngine(file_name)

this was no DB file
'str' object has no attribute 'read'
DKB Mapping successful: No error found..
DKB data mapped!
[INFO] enrich data done!
2022-01-11 15:09:14.543120
Loading model..
2022-01-11 15:09:16.651469
Predicting categories..
0    2019-01-22
1    2019-01-21
2    2019-01-21
3    2019-01-18
4    2019-01-17
        ...    
74   2018-12-27
75   2018-12-27
76   2018-12-27
77   2018-12-27
78   2018-12-27
Name: Buchungstag, Length: 79, dtype: datetime64[ns]
{(2018, 12): [64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78], (2019, 1): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]}
overall income:2313.7
income: 1156.85
Computing income indicator..
expenses: 518.0
Computing expense indicator..
Purchasing Power: 638.8499999999999
vec
['Income', 'Purchasing Power', 'Expenses']
2.0
exp

In [7]:
dataNode.tmp_result

'{"metadata": {"status": "OK", "processing_date": "03:09PM on January 11, 2022"}, "indicators": {"purchasing_power": {"label": "Purchasing Power", "value": "638.8499999999999", "type": "numeric", "description": "Balance available after deduction of monthly expenses.", "property": "purchasing power", "bins": [0.0, 801.2499999999999, 1602.4999999999998, 2403.7499999999995, 3204.9999999999995, 4006.2499999999995, 4807.499999999999, 5608.749999999999, 6409.999999999999, 7211.249999999999, 8012.499999999999], "values": [74, 6, 8, 3, 2, 1, 0, 0, 1, 1], "mu": 897.8499999999999, "std": 3108.3999999999996, "quantile": 0.0, "relevance": 1.0}, "income": {"label": "Income", "value": "1156.85", "type": "numeric", "description": "Monthly net income", "property": "income", "bins": [0.0, 900.0949999999999, 1800.1899999999998, 2700.285, 3600.3799999999997, 4500.474999999999, 5400.57, 6300.664999999999, 7200.759999999999, 8100.855, 9000.949999999999], "values": [75, 8, 5, 7, 1, 0, 0, 0, 1, 0], "mu": 115

In [8]:
dataNode.result

'{"metadata": {"status": "OK", "processing_date": "03:09PM on January 11, 2022"}, "report": {"url": "http://www.google.de"}, "ui": {"Summary-Dashboard": {"summary-dashboard-chart-1": {"info": {"name": "vec", "description": "Pretty neat features", "type": "vector"}, "data": [{"type": "scatterpolar", "r": [0.5, 0.5, 0.5], "theta": ["Income", "Purchasing Power", "Expenses"], "fill": "toself", "linewidth": 4, "hoverinfo": "name+text", "hovertext": ["Quantile: 49%", "Quantile: 50%", "Quantile: 58%", "Quantile: 47%", "Quantile: 48%", "Quantile: 49%", "Quantile: 50%"], "name": "Peer group"}, {"type": "scatterpolar", "r": [0.7731958762886598, 0.0, 1.0], "theta": ["Income", "Purchasing Power", "Expenses"], "fill": "toself", "linewidth": 4, "hoverinfo": "name+text", "hovertext": ["Quantile: 49%", "Quantile: 50%", "Quantile: 58%", "Quantile: 47%", "Quantile: 48%", "Quantile: 49%", "Quantile: 50%"], "name": "Your spectrum"}], "layout": {"polar": {"radialaxis": {"visible": true, "range": [0, 1.2], 

In [10]:
with open("input.txt","w",encoding='utf-8') as f:
    f.write(categorizer.output)

### API
- cs_get_data GET
    - function: cs_get_data()
    - ?enableCORS=true
- cs_upload POST
    - cs_data_receiver()
    - authorization
    - accept-encoding
    - ?enableMulti-partFilter=false&amp;@enableCORS=true&amp;disableStreamCache=false

In [9]:


def cs_get_data():
    res=dataNode.result
    if(res!=""):
        return res
    else:
        return "computing"
    
def cs_data_receiver():
    from base64 import b64decode
    import json
    import time


    dts = time.strftime('%Y-%m-%d %H:%M:%S')

    res = { }

    encoded_content=request.getContent()

    pltool.printLine("invoked joern route at "+dts)

    if encoded_content is None:
        encoded_content = "Content:Ok"

    content=encoded_content
    res["headers"]=request.getHeaders()
    res["parameters"]=request.getParameters()
    res["output"]=content

    #### new code to run the Commonsign Engine
    pltool.printLine('input')
    pltool.printLine(content)
    json_content=json.loads(str(content).replace("\'","\""))
    blob=b64decode(json_content["filedata"]).decode("ISO-8859-1")

    id=libs.getLibObject("workspaces.CommonSign")
    node=libs.getLibObject("workspaces.CommonSign.Core_Engine")
    pltool.setContextVar(id,"input",blob)
    pltool.evaluateNode("#14:0",node)


    return res

In [11]:
from flask import Flask, jsonify, request
import json

app = Flask(__name__)

@app.route("/cs_get_data/",methods=["GET"])
def get_data():
    return jsonify(cs_get_data())

@app.rout("/cs_upload/",methods=['POST'])
def test():
    res = json.loads(request.data)
    return jsonify("success")

app.run(host="0.0.0.0",port=8080)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://10.82.5.76:105/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Jan/2022 15:10:07] "GET /cs_get_data/ HTTP/1.1" 200 -
127.0.0.1 - - [11/Jan/2022 15:10:08] "GET /cs_get_data/ HTTP/1.1" 200 -
127.0.0.1 - - [11/Jan/2022 15:10:10] "GET /cs_get_data/ HTTP/1.1" 200 -
127.0.0.1 - - [11/Jan/2022 15:10:12] "GET /cs_get_data/ HTTP/1.1" 200 -


# PLAYGROUND

In [4]:
import sklearn
sklearn.__version__

'0.19.1'

In [3]:
inputstring = '''"Kontonummer:";"DE38120300001036245114 / Girokonto"; "Von:";"24.12.2018"; "Bis:";"23.01.2019"; "Kontostand vom 23.01.2019:";"477,93 EUR"; "Buchungstag";"Wertstellung";"Buchungstext";"Auftraggeber / Beguenstigter";"Verwendungszweck";"Kontonummer";"BLZ";"Betrag (EUR)";"Glaeubiger-ID";"Mandatsreferenz";"Kundenreferenz"; "22.01.2019";"22.01.2019";"Lastschrift";"PayPal Europe S.a.r.l. et Cie S.C.A";"PP.9674.PP . LIEFERHELD, Ihr Einkauf bei LIEFERHELD AWV-MELDEPFLICHT BEACHTEN HOTLINE BUNDESBANK: (0800) 1234-111";"FR7630004021180001013711792";"BNPAFRPPXXX";"-12,40";"LU96ZZZ0000000000000000058 ";"4EEJ224PC6BAL ";""; "21.01.2019";"21.01.2019";"Geldautomat";"Potsdam//Potsdam/DE / SPARDA-BANK BERLIN";"2019-01-18T19:03 Debitk.1 2020-12 Fremdentgelt 4,90 EUR";"DE31120965972004061500";"GENODEF1S10";"-154,90";"";"";"00001056025337180119190321"; "21.01.2019";"21.01.2019";"Kartenzahlung/-abrechnung";"EDEKA 2372//POTSDAM/DE / EDEKA 2372";"2019-01-19T11:42 Debitk.1 2020-12";"DE03200907002535050071";"EDEKDEHHXXX";"-39,67";"";"";"58016060008607190119114212"; "18.01.2019";"18.01.2019";"Kartenzahlung/-abrechnung";"EDEKA 2372//POTSDAM/DE / EDEKA 2372";"2019-01-17T19:56 Debitk.1 2020-12";"DE03200907002535050071";"EDEKDEHHXXX";"-9,76";"";"";"58016060008131170119195651"; "17.01.2019";"17.01.2019";"Kartenzahlung/-abrechnung";"Burger King Store 25460 Frankfurt S//FRANKFURT/DE / Burger King Store 25460 Frankfurt S";"2019-01-15T17:36 Debitk.1 2020-12";"DE03500800000096743900";"DRESDEFFXXX";"-9,14";"";"";"52080888001245150119173604"; "17.01.2019";"17.01.2019";"Lastschrift";"PayPal Europe S.a.r.l. et Cie S.C.A";"PP.9674.PP . DBVERTRIEBG, Ihr Einkauf bei DBVERTRIEBG AWV-MELDEPFLICHT BEACHTEN HOTLINE BUNDESBANK: (0800) 1234-111";"FR7630004021180001013711792";"BNPAFRPPXXX";"-4,50";"LU96ZZZ0000000000000000058 ";"4EEJ224PC6BAL ";""; "17.01.2019";"17.01.2019";"Lastschrift";"PayPal Europe S.a.r.l. et Cie S.C.A";"PP.9674.PP . DBVERTRIEBG, Ihr Einkauf bei DBVERTRIEBG AWV-MELDEPFLICHT BEACHTEN HOTLINE BUNDESBANK: (0800) 1234-111";"FR7630004021180001013711792";"BNPAFRPPXXX";"-4,95";"LU96ZZZ0000000000000000058 ";"4EEJ224PC6BAL ";""; "16.01.2019";"16.01.2019";"Lastschrift";"PayPal Europe S.a.r.l. et Cie S.C.A";"PP.9674.PP . DBVERTRIEBG, Ihr Einkauf bei DBVERTRIEBG AWV-MELDEPFLICHT BEACHTEN HOTLINE BUNDESBANK: (0800) 1234-111";"FR7630004021180001013711792";"BNPAFRPPXXX";"-4,95";"LU96ZZZ0000000000000000058 ";"4EEJ224PC6BAL ";""; "15.01.2019";"15.01.2019";"Kartenzahlung/-abrechnung";"AAREAL BANK AG//WIESBADEN/DE / AAREAL BANK AG";"2019-01-14T11:53 Debitk.1 2020-12";"DE40664926000009104216";"GENODE61APP";"-50,00";"";"";"54147561333645140119115356"; "15.01.2019";"15.01.2019";"Lastschrift";"PayPal Europe S.a.r.l. et Cie S.C.A";"PP.9674.PP . SPOTIFY, Ihr Einkauf bei SPOTIFY AWV-MELDEPFLICHT BEACHTEN HOTLINE BUNDESBANK: (0800) 1234-111";"FR7630004021180001013711792";"BNPAFRPPXXX";"-14,99";"LU96ZZZ0000000000000000058 ";"4EEJ224PC6BAL ";""; "15.01.2019";"15.01.2019";"Lastschrift";"PayPal Europe S.a.r.l. et Cie S.C.A";"PP.9674.PP . active-licence, Ihr Einkauf bei active-licence, Artikel-273643788482 AWV-MELDEPFLICHT BEACHTEN HOTLINE BUNDESBANK: (0800) 1234-111";"FR7630004021180001013711792";"BNPAFRPPXXX";"-14,49";"LU96ZZZ0000000000000000058 ";"4EEJ224PC6BAL ";""; "15.01.2019";"15.01.2019";"Kartenzahlung/-abrechnung";"REWE SAGT DANKE. 44401516//Wiesbaden/DE / REWE Markt GmbH";"2019-01-14T21:24 Debitk.1 2020-12";"DE10700202700015820758";"HYVEDEMMXXX";"-5,52";"";"";"56001982331784140119212419"; "15.01.2019";"15.01.2019";"Kartenzahlung/-abrechnung";"MONDO T2 REST. GIR 69279283//BERLIN/DE / MONDO T2 REST.";"2019-01-14T05:30 Debitk.1 2020-12";"DE51300500000001474915";"WELADEDDXXX";"-3,90";"";"";"69279283009642140119053041"; "14.01.2019";"14.01.2019";"Kartenzahlung/-abrechnung";"DM FIL.2740 H:65128//POTSDAM/DE / DM DROGERIEMARKT SAGT DANKE";"2019-01-12T11:10 Debitk.1 2020-12";"DE88600501017402099997";"SOLADEST600";"-30,40";"";"";"65128269006173120119111029"; "14.01.2019";"14.01.2019";"ï¿½berweisung";"ADYEN B.V.";"1615472935603032 Opodo Opodo-4106868705 142591-358975-5C39D378-8849 DATUM 12.01.2019, 12.48 UHR1.TAN 632946";"DE25700111105100580006";"DEKTDE7GXXX";"-192,17";"";"";"NOTPROVIDED"; "14.01.2019";"14.01.2019";"Lastschrift";"PayPal Europe S.a.r.l. et Cie S.C.A";"PP.9674.PP . JACKWOLFSKI, Ihr Einkauf bei JACKWOLFSKI AWV-MELDEPFLICHT BEACHTEN HOTLINE BUNDESBANK: (0800) 1234-111";"FR7630004021180001013711792";"BNPAFRPPXXX";"-82,90";"LU96ZZZ0000000000000000058 ";"4EEJ224PC6BAL ";""; "14.01.2019";"14.01.2019";"Gutschrift";"JOERN MALICH";"";"DE05120300001052729694";"BYLADEM1001";"600,00";"";"";"NOTPROVIDED"; "14.01.2019";"14.01.2019";"Kartenzahlung/-abrechnung";"EDEKA 2372//POTSDAM/DE / EDEKA 2372";"2019-01-12T11:38 Debitk.1 2020-12";"DE03200907002535050071";"EDEKDEHHXXX";"-53,88";"";"";"58016075004572120119113811"; "14.01.2019";"14.01.2019";"ï¿½berweisung";"ADYEN B.V.";"1315472927957886 Opodo Opodo-4106830966 142591-358975-5C39D07B-549D DATUM 12.01.2019, 12.35 UHR1.TAN 624663";"DE25700111105100580006";"DEKTDE7GXXX";"-231,24";"";"";"NOTPROVIDED"; "14.01.2019";"14.01.2019";"Lastschrift";"1u1 Telecom GmbH";"KD-Nr. K449320431/ RG-Nr. 150502958346";"DE97500400000589011600";"COBADEFFXXX";"-43,35";"DE77ZZZ00000569719 ";"0200010224440 ";""; "11.01.2019";"11.01.2019";"Lastschrift";"PayPal Europe S.a.r.l. et Cie S.C.A";"PP.9674.PP . DBVERTRIEBG, Ihr Einkauf bei DBVERTRIEBG AWV-MELDEPFLICHT BEACHTEN HOTLINE BUNDESBANK: (0800) 1234-111";"FR7630004021180001013711792";"BNPAFRPPXXX";"-4,95";"LU96ZZZ0000000000000000058 ";"4EEJ224PC6BAL ";""; "11.01.2019";"11.01.2019";"Kartenzahlung/-abrechnung";"EDEKA 2372//POTSDAM/DE / EDEKA 2372";"2019-01-10T18:11 Debitk.1 2020-12";"DE03200907002535050071";"EDEKDEHHXXX";"-19,06";"";"";"58016058005467100119181130"; "11.01.2019";"11.01.2019";"Lastschrift";"PayPal Europe S.a.r.l. et Cie S.C.A";"PP.9674.PP . MYTAXI, Ihr Einkauf bei MYTAXI AWV-MELDEPFLICHT BEACHTEN HOTLINE BUNDESBANK: (0800) 1234-111";"FR7630004021180001013711792";"BNPAFRPPXXX";"-6,99";"LU96ZZZ0000000000000000058 ";"4EEJ224PC6BAL ";""; "10.01.2019";"10.01.2019";"Kartenzahlung/-abrechnung";"VAPIANO WIESBADEN//Wiesbad/DE / VAPIANO WIESBADEN";"2019-01-09T12:20 Debitk.1 2020-12";"DE85510500150107080046";"NASSDE55XXX";"-14,70";"";"";"55614831109914090119122017"; "10.01.2019";"10.01.2019";"Lastschrift";"AMAZON PAYMENTS EUROPE S.C.A.";"303-9636940-8227547 AMZN Mktp DE 1301523509816361";"DE87300308801908262006";"TUBDDEDD";"-6,84";"DE94ZZZ00000561653 ";"S?m+)43Kl5g8B0tKOkg9QTJ:+qIqcG ";""; "10.01.2019";"10.01.2019";"Lastschrift";"PayPal Europe S.a.r.l. et Cie S.C.A";"PP.9674.PP . DBVERTRIEBG, Ihr Einkauf bei DBVERTRIEBG AWV-MELDEPFLICHT BEACHTEN HOTLINE BUNDESBANK: (0800) 1234-111";"FR7630004021180001013711792";"BNPAFRPPXXX";"-6,45";"LU96ZZZ0000000000000000058 ";"4EEJ224PC6BAL ";""; "10.01.2019";"10.01.2019";"Lastschrift";"AMAZON PAYMENTS EUROPE S.C.A.";"303-6245864-6056361 AMZN Mktp DE 1465750303841137";"DE87300308801908262006";"TUBDDEDD";"-19,99";"DE94ZZZ00000561653 ";"S?m+)43Kl5g8B0tKOkg9QTJ:+qIqcG ";""; "09.01.2019";"09.01.2019";"Kartenzahlung/-abrechnung";"MONDO T2 REST. GIR 69279283//BERLIN/DE / MONDO T2 REST.";"2019-01-08T06:31 Debitk.1 2020-12";"DE51300500000001474915";"WELADEDDXXX";"-3,90";"";"";"69279283009093080119063130"; "09.01.2019";"09.01.2019";"Kartenzahlung/-abrechnung";"REWE SAGT DANKE. 44401516//Wiesbaden/DE / REWE Markt GmbH";"2019-01-08T20:32 Debitk.1 2020-12";"DE10700202700015820758";"HYVEDEMMXXX";"-3,03";"";"";"56001981331121080119203233"; "09.01.2019";"09.01.2019";"Lastschrift";"Christin Seeger";"F-last., DAS Tanzhaus Potsdam G-ID:DE55ZZZ00001720632, Ref.:001361-001523 Grund: Beitrag 1 TH-Club Fr";"DE86100900002541330006";"BEVODEBB";"-36,00";"DE55ZZZ00001720632 ";"001361-001523 ";""; "09.01.2019";"09.01.2019";"Lastschrift";"Christin Seeger";"F-last., DAS Tanzhaus Potsdam G-ID:DE55ZZZ00001720632, Ref.:001362-001522 Grund: Beitrag 1 TH-Club Fr";"DE86100900002541330006";"BEVODEBB";"-36,00";"DE55ZZZ00001720632 ";"001362-001522 ";""; "09.01.2019";"09.01.2019";"Kartenzahlung/-abrechnung";"VAPIANO WIESBADEN//Wiesbad/DE / VAPIANO WIESBADEN";"2019-01-08T20:21 Debitk.1 2020-12";"DE85510500150107080046";"NASSDE55XXX";"-16,00";"";"";"55614831109882080119202100"; "09.01.2019";"09.01.2019";"Kartenzahlung/-abrechnung";"Berliner Pilsner Bar//Berlin/DE / ick bin ein berliner";"2019-01-08T06:59 Debitk.1 2020-12";"DE31100208900014922466";"HYVEDEMM488";"-3,60";"";"";"61297961147674080119065926"; "09.01.2019";"09.01.2019";"Lastschrift";"AMAZON PAYMENTS EUROPE S.C.A.";"303-6245864-6056361 AMZN Mktp DE 1575032168137089";"DE87300308801908262006";"TUBDDEDD";"-19,99";"DE94ZZZ00000561653 ";"S?m+)43Kl5g8B0tKOkg9QTJ:+qIqcG ";""; "09.01.2019";"09.01.2019";"Lastschrift";"PayPal Europe S.a.r.l. et Cie S.C.A";"PP.9674.PP . TAKEAWAYCOM, Ihr Einkauf bei TAKEAWAYCOM AWV-MELDEPFLICHT BEACHTEN HOTLINE BUNDESBANK: (0800) 1234-111";"FR7630004021180001013711792";"BNPAFRPPXXX";"-12,30";"LU96ZZZ0000000000000000058 ";"4EEJ224PC6BAL ";""; "08.01.2019";"08.01.2019";"Lastschrift";"PayPal Europe S.a.r.l. et Cie S.C.A";"PP.9674.PP . , Ihr Einkauf bei AWV-MELDEPFLICHT BEACHTEN HOTLINE BUNDESBANK: (0800) 1234-111";"FR7630004021180001013711792";"BNPAFRPPXXX";"-80,00";"LU96ZZZ0000000000000000058 ";"4EEJ224PC6BAL ";""; "08.01.2019";"08.01.2019";"Kartenzahlung/-abrechnung";"EDEKA 2372//POTSDAM/DE / EDEKA 2372";"2019-01-07T17:37 Debitk.1 2020-12";"DE03200907002535050071";"EDEKDEHHXXX";"-24,59";"";"";"58016056001547070119173700"; "08.01.2019";"08.01.2019";"Dauerauftrag";"DKB VISACARD";"4930000023122416 18.18 JOERN MALICH DKB INTERNET BANKING";"DE63120300000001999333";"BYLADEM1001";"-200,00";"";"";"NOTPROVIDED"; "07.01.2019";"07.01.2019";"ï¿½berweisung";"ADYEN B.V.";"4315467767739554 Opodo Opodo-4089607336 142591-358975-5C31F0C5-EEF3 DATUM 06.01.2019, 13.15 UHR1.TAN 628438";"DE25700111105100580006";"DEKTDE7GXXX";"-200,87";"";"";"NOTPROVIDED"; "07.01.2019";"07.01.2019";"Dauerauftrag";"DKB VISACARD";"4930000022666520 12.36 JOERN MALICH DKB INTERNET BANKING";"DE63120300000001999333";"BYLADEM1001";"-400,00";"";"";"NOTPROVIDED"; "07.01.2019";"07.01.2019";"Dauerauftrag";"HERR JOERN MALICH";"Notreserven";"DE05120300001052729694";"BYLADEM1001";"-200,00";"";"";"NOTPROVIDED"; "07.01.2019";"07.01.2019";"Kartenzahlung/-abrechnung";"BEROLINA APOTHEKE//BERLIN/DE / BEROLINA APOTHEKE";"2019-01-04T11:18 Debitk.1 2020-12";"DE75300606010003984926";"DAAEDEDDXXX";"-13,70";"";"";"65379449017064040119111808"; "07.01.2019";"07.01.2019";"Lastschrift";"SOS-Kinderdorf e.V.";"0423728351 1/19 10.00";"DE28700400410151300100";"COBADEFFXXX";"-10,00";"DE61ZZZ00000002363 ";"3892916 ";""; "04.01.2019";"04.01.2019";"Kartenzahlung/-abrechnung";"EDEKA 2372//POTSDAM/DE / EDEKA 2372";"2019-01-02T17:14 Debitk.1 2020-12";"DE03200907002535050071";"EDEKDEHHXXX";"-46,18";"";"";"58016060005055020119171458"; "04.01.2019";"04.01.2019";"Gutschrift";"PAYPAL EUROPE SARL ET CIE SCA";"PP.9674.PP ABBUCHUNG VOM PAYPAL-KONTO AWV-MELDEPFLICHT BEACHTEN HOTLINE BUNDESBANK: (0800) 1234-111";"FR7630004021180001013711792";"BNPAFRPP";"38,00";"";"";"YYW4EEJ2ANV87KPC PP.9674.PP PAYPAL"; "04.01.2019";"04.01.2019";"Kartenzahlung/-abrechnung";"EDEKA 2372//POTSDAM/DE / EDEKA 2372";"2019-01-03T20:59 Debitk.1 2020-12";"DE03200907002535050071";"EDEKDEHHXXX";"-7,21";"";"";"58016060005356030119205927"; "04.01.2019";"04.01.2019";"Lastschrift";"KARSTADT WARENHAUS GMBH";"020111391011568201201324140 KARSTADT BEDANKT SICH";"DE69300308800301859007";"TUBDDEDDXXX";"-31,49";"DE65ZZZ00000258565 ";"6021156841901901021139";""; "04.01.2019";"04.01.2019";"Lastschrift";"AMAZON EU S.A R.L., NIEDERLASSUNG DEUTSCHLAND";"305-1919901-3905137 Amazon.de 7328996508555891";"DE93300308800013441006";"TUBDDEDD";"-9,99";"DE24ZZZ00000561652 ";"S?m+)43Kl5g8B0tKOkg9QTJ:+qIqcG ";""; "04.01.2019";"04.01.2019";"Gutschrift";"Sophie Schmidt";"Tanzhaus";"DE95160500001101611452";"WELADED1PMB";"36,00";"";"";"NOTPROVIDED"; "04.01.2019";"04.01.2019";"Lastschrift";"East Bank Club The Fitness Factory GmbH";"IdentNr:31-119296/ Superfit Mitgliedsbeitrag Jan19 Potsdam";"DE43100900007347340019";"BEVODEBB";"-18,50";"DE52ZZZ00000094093 ";"7-7014513 ";""; "03.01.2019";"03.01.2019";"Kartenzahlung/-abrechnung";"MY KENG//Potsdam/DE / MY KENG";"2019-01-02T12:28 Debitk.1 2020-12";"DE29120700240303786800";"DEUTDEDB160";"-31,70";"";"";"55564852142750020119122859"; "03.01.2019";"03.01.2019";"Lastschrift";"PayPal Europe S.a.r.l. et Cie S.C.A";"PP.9674.PP . TAKEAWAYCOM, Ihr Einkauf bei TAKEAWAYCOM AWV-MELDEPFLICHT BEACHTEN HOTLINE BUNDESBANK: (0800) 1234-111";"FR7630004021180001013711792";"BNPAFRPPXXX";"-46,30";"LU96ZZZ0000000000000000058 ";"4EEJ224PC6BAL ";""; "02.01.2019";"02.01.2019";"Lastschrift";"PayPal Europe S.a.r.l. et Cie S.C.A";"PP.9674.PP . UCIKINOWELT, Ihr Einkauf bei UCIKINOWELT AWV-MELDEPFLICHT BEACHTEN HOTLINE BUNDESBANK: (0800) 1234-111";"FR7630004021180001013711792";"BNPAFRPPXXX";"-10,50";"LU96ZZZ0000000000000000058 ";"4EEJ224PC6BAL ";"1004606973648 PP.9674.PP PAYPAL"; "02.01.2019";"02.01.2019";"Lastschrift";"PayPal Europe S.a.r.l. et Cie S.C.A";"PP.9674.PP . PLAYSTATION, Ihr Einkauf bei PLAYSTATION AWV-MELDEPFLICHT BEACHTEN HOTLINE BUNDESBANK: (0800) 1234-111";"FR7630004021180001013711792";"BNPAFRPPXXX";"-7,99";"LU96ZZZ0000000000000000058 ";"4EEJ224PC6BAL ";""; "02.01.2019";"02.01.2019";"Kartenzahlung/-abrechnung";"THALIA FILMTHEATER GIR 79981695//POTSDAM/DE / THALIA FILMTHEATER";"2018-12-29T01:22 Debitk.1 2020-12";"DE19300500000001458710";"WELADEDDXXX";"-39,20";"";"";"79981695014892291218012236"; "02.01.2019";"02.01.2019";"Lastschrift";"Neue Bayerische Beamten Lebensversicherung AG";"Zahlbelegnr. 426148168 Vers.Nr. B090390255907 / Beitrag EUR 84.94 die Bayerische ABWA+Neue Bay.Beamten Lebensv.";"DE61700500000003040745";"BYLADEMMXXX";"-84,94";"DE97ZZZ00000106475 ";"B090390255907 ";""; "02.01.2019";"02.01.2019";"Gutschrift";"PRAGMALAB GMBH";"Gehalt 12/2018";"DE34100208900601848848";"HYVEDEMM488";"2.313,70";"";"";"NOTPROVIDED"; "02.01.2019";"02.01.2019";"Gutschrift";"PRAGMALAB GMBH";"Reisekostenabrechnung 11/2018, 12/2018";"DE34100208900601848848";"HYVEDEMM488";"863,50";"";"";"NOTPROVIDED"; "02.01.2019";"02.01.2019";"Lastschrift";"S-BAHN BERLIN GMBH";"";"DE86100100100146915107";"PBNKDEFFXXX";"-82,67";"DE3620R00000002134 ";"43692764-1 ";""; "02.01.2019";"02.01.2019";"Lastschrift";"Neue Bayerische Beamten Lebensversicherung AG";"Zahlbelegnr. 426148176 Vers.Nr. B090390255907 / Beitrag EUR 84.94 die Bayerische ABWA+Neue Bay.Beamten Lebensv.";"DE61700500000003040745";"BYLADEMMXXX";"-84,94";"DE97ZZZ00000106475 ";"B090390255907 ";""; "02.01.2019";"02.01.2019";"Lastschrift";"PayPal Europe S.a.r.l. et Cie S.C.A";"PP.9674.PP . UCIKINOWELT, Ihr Einkauf bei UCIKINOWELT AWV-MELDEPFLICHT BEACHTEN HOTLINE BUNDESBANK: (0800) 1234-111";"FR7630004021180001013711792";"BNPAFRPPXXX";"-38,00";"LU96ZZZ0000000000000000058 ";"4EEJ224PC6BAL ";"1004605049928 PP.9674.PP PAYPAL"; "02.01.2019";"02.01.2019";"Dauerauftrag";"THOMAS WEMBACHER";"MIETE MIT STROMKOSTEN";"DE58160500004503386986";"WELADED1PMB";"-518,00";"";"";"NOTPROVIDED"; "02.01.2019";"02.01.2019";"Lastschrift";"PayPal Europe S.a.r.l. et Cie S.C.A";"PP.9674.PP . DBVERTRIEBG, Ihr Einkauf bei DBVERTRIEBG AWV-MELDEPFLICHT BEACHTEN HOTLINE BUNDESBANK: (0800) 1234-111";"FR7630004021180001013711792";"BNPAFRPPXXX";"-24,00";"LU96ZZZ0000000000000000058 ";"4EEJ224PC6BAL ";"1004608166898 PP.9674.PP PAYPAL"; "02.01.2019";"02.01.2019";"Lastschrift";"PayPal Europe S.a.r.l. et Cie S.C.A";"PP.9674.PP . DBVERTRIEBG, Ihr Einkauf bei DBVERTRIEBG AWV-MELDEPFLICHT BEACHTEN HOTLINE BUNDESBANK: (0800) 1234-111";"FR7630004021180001013711792";"BNPAFRPPXXX";"-11,00";"LU96ZZZ0000000000000000058 ";"4EEJ224PC6BAL ";"1004619854333 PP.9674.PP PAYPAL"; "28.12.2018";"28.12.2018";"Gutschrift";"Sophie Schmidt";"Miete";"DE95160500001101611452";"WELADED1PMB";"225,00";"";"";"NOTPROVIDED"; "28.12.2018";"28.12.2018";"Kartenzahlung/-abrechnung";"DM FIL.2740 H:65128//POTSDAM/DE / DM DROGERIEMARKT SAGT DANKE";"2018-12-27T12:55 Debitk.1 2020-12";"DE88600501017402099997";"SOLADEST600";"-16,90";"";"";"65128268004156271218125556"; "28.12.2018";"01.01.2019";"Abschluss";"";"Abrechnung 28.12.2018 siehe Anlage<br />Abrechnung 28.12.2018<br />Information zur Abrechnung<br />Kontostand am 28.12.2018 21,47 +<br />Abrechnungszeitraum vom 01.10.2018 bis 31.12.2018<br />Zinsen fï¿½r eingerï¿½umte Kontoï¿½berziehung 0,06-<br /> 6,9000 v.H. Kred-Zins bis 03.12.2018<br />Abrechnung 31.12.2018 0,06-<br />Sollzinssï¿½tze am 30.12.2018<br /> 6,9000 v.H. fï¿½r eingerï¿½umte Kontoï¿½berziehung<br />(aktuell eingerï¿½umte Kontoï¿½berziehung 500,00)<br /> 6,9000 v.H. fï¿½r geduldete Kontoï¿½berziehung<br />ï¿½ber die eingerï¿½umte Kontoï¿½berziehung hinaus<br />Kontostand/Rechnungsabschluss am 28.12.2018 21,41 +<br />Rechnungsnummer: 20181228-BY111-00057038820";"1036245114";"12030000";"-0,06";"";"";""; "28.12.2018";"28.12.2018";"";"KREDITKARTENABRECHNUNG";"VISA-ABR. 493000XXXXXX9098";"9003290294";"12030000";"-300,00";"";"";""; "27.12.2018";"27.12.2018";"Kartenzahlung/-abrechnung";"DM FIL.2740 H:65128//POTSDAM/DE / DM DROGERIEMARKT SAGT DANKE";"2018-12-22T12:12 Debitk.1 2020-12";"DE88600501017402099997";"SOLADEST600";"-3,00";"";"";"65128268003702221218121212"; "27.12.2018";"27.12.2018";"Kartenzahlung/-abrechnung";"EDEKA 2372//POTSDAM/DE / EDEKA 2372";"2018-12-23T17:19 Debitk.1 2020-12";"DE03200907002535050071";"EDEKDEHHXXX";"-15,68";"";"";"58016060003806231218171945"; "27.12.2018";"27.12.2018";"Lastschrift";"PayPal Europe S.a.r.l. et Cie S.C.A";"PP.9674.PP . TAKEAWAYCOM, Ihr Einkauf bei TAKEAWAYCOM AWV-MELDEPFLICHT BEACHTEN HOTLINE BUNDESBANK: (0800) 1234-111";"FR7630004021180001013711792";"BNPAFRPPXXX";"-37,90";"LU96ZZZ0000000000000000058 ";"4EEJ224PC6BAL ";"1004562939730 PP.9674.PP PAYPAL"; "27.12.2018";"27.12.2018";"Kartenzahlung/-abrechnung";"Kaufland Potsdam//Potsdam/DE / KAUFLAND";"2018-12-24T09:13 Debitk.1 2020-12";"DE25600501010004001050";"SOLADEST600";"-0,35";"";"";"61327682117627241218091339"; "27.12.2018";"27.12.2018";"Kartenzahlung/-abrechnung";"EDEKA 2372//POTSDAM/DE / EDEKA 2372";"2018-12-22T11:47 Debitk.1 2020-12";"DE03200907002535050071";"EDEKDEHHXXX";"-78,14";"";"";"58016075002246221218114732"; "27.12.2018";"27.12.2018";"Kartenzahlung/-abrechnung";"Kaufland Potsdam//Potsdam/DE / KAUFLAND";"2018-12-24T09:07 Debitk.1 2020-12";"DE25600501010004001050";"SOLADEST600";"-4,38";"";"";"61327685068888241218090734"; "27.12.2018";"27.12.2018";"Lastschrift";"PayPal Europe S.a.r.l. et Cie S.C.A";"PP.9674.PP . PLAYSTATION, Ihr Einkauf bei PLAYSTATION AWV-MELDEPFLICHT BEACHTEN HOTLINE BUNDESBANK: (0800) 1234-111";"FR7630004021180001013711792";"BNPAFRPPXXX";"-17,99";"LU96ZZZ0000000000000000058 ";"4EEJ224PC6BAL ";"1004569810505 PP.9674.PP PAYPAL"; "27.12.2018";"27.12.2018";"Kartenzahlung/-abrechnung";"DM FIL.2740 H:65128//POTSDAM/DE / DM DROGERIEMARKT SAGT DANKE";"2018-12-22T12:06 Debitk.1 2020-12";"DE88600501017402099997";"SOLADEST600";"-11,10";"";"";"65128268003696221218120652"; "27.12.2018";"27.12.2018";"Lastschrift";"PayPal Europe S.a.r.l. et Cie S.C.A";"PP.9674.PP . GREYZONE, Ihr Einkauf bei GREYZONE AWV-MELDEPFLICHT BEACHTEN HOTLINE BUNDESBANK: (0800) 1234-111";"FR7630004021180001013711792";"BNPAFRPPXXX";"-115,00";"LU96ZZZ0000000000000000058 ";"4EEJ224PC6BAL ";"1004567744595 PP.9674.PP PAYPAL"; "27.12.2018";"27.12.2018";"Kartenzahlung/-abrechnung";"EDEKA 2372//POTSDAM/DE / EDEKA 2372";"2018-12-22T10:49 Debitk.1 2020-12";"DE03200907002535050071";"EDEKDEHHXXX";"-6,56";"";"";"58019489000132221218104948"; "27.12.2018";"27.12.2018";"Kartenzahlung/-abrechnung";"Aral Tankstelle Potsdam Grossbeerenstrase 121//Potsdam/DE / ARAL AG";"Kartenzahlung 2018-12-25T19:39 Debitk.1 2020-12";"DE95430800830802029200";"DRESDEFF430";"-8,97";"";"";"71015812499393251218193900";'''

In [5]:
pd.read_csv(StringIO(inputstring))

,"Kontonummer:;""DE38120300001036245114 / Girokonto""; ""Von:"";""24.12.2018""; ""Bis:"";""23.01.2019""; ""Kontostand vom 23.01.2019:"";""477","93 EUR""; ""Buchungstag"";""Wertstellung"";""Buchungstext"";""Auftraggeber / Beguenstigter"";""Verwendungszweck"";""Kontonummer"";""BLZ"";""Betrag (EUR)"";""Glaeubiger-ID"";""Mandatsreferenz"";""Kundenreferenz""; ""22.01.2019"";""22.01.2019"";""Lastschrift"";""PayPal Europe S.a.r.l. et Cie S.C.A"";""PP.9674.PP . LIEFERHELD","Ihr Einkauf bei LIEFERHELD AWV-MELDEPFLICHT BEACHTEN HOTLINE BUNDESBANK: (0800) 1234-111"";""FR7630004021180001013711792"";""BNPAFRPPXXX"";""-12","40"";""LU96ZZZ0000000000000000058 "";""4EEJ224PC6BAL "";""""; ""21.01.2019"";""21.01.2019"";""Geldautomat"";""Potsdam//Potsdam/DE / SPARDA-BANK BERLIN"";""2019-01-18T19:03 Debitk.1 2020-12 Fremdentgelt 4","90 EUR"";""DE31120965972004061500"";""GENODEF1S10"";""-154","90"";"""";"""";""00001056025337180119190321""; ""21.01.2019"";""21.01.2019"";""Kartenzahlung/-abrechnung"";""EDEKA 2372//POTSDAM/DE / EDEKA 2372"";""2019-01-19T11:42 Debitk.1 2020-12"";""DE03200907002535050071"";""EDEKDEHHXXX"";""-39","67"";"""";"""";""58016060008607190119114212""; ""18.01.2019"";""18.01.2019"";""Kartenzahlung/-abrechnung"";""EDEKA 2372//POTSDAM/DE / EDEKA 2372"";""2019-01-17T19:56 Debitk.1 2020-12"";""DE03200907002535050071"";""EDEKDEHHXXX"";""-9","76"";"""";"""";""58016060008131170119195651""; ""17.01.2019"";""17.01.2019"";""Kartenzahlung/-abrechnung"";""Burger King Store 25460 Frankfurt S//FRANKFURT/DE / Burger King Store 25460 Frankfurt S"";""2019-01-15T17:36 Debitk.1 2020-12"";""DE03500800000096743900"";""DRESDEFFXXX"";""-9","14"";"""";"""";""52080888001245150119173604""; ""17.01.2019"";""17.01.2019"";""Lastschrift"";""PayPal Europe S.a.r.l. et Cie S.C.A"";""PP.9674.PP . DBVERTRIEBG","Ihr Einkauf bei DBVERTRIEBG AWV-MELDEPFLICHT BEACHTEN HOTLINE BUNDESBANK: (0800) 1234-111"";""FR7630004021180001013711792"";""BNPAFRPPXXX"";""-4",...,"35"";"""";"""";""61327682117627241218091339""; ""27.12.2018"";""27.12.2018"";""Kartenzahlung/-abrechnung"";""EDEKA 2372//POTSDAM/DE / EDEKA 2372"";""2018-12-22T11:47 Debitk.1 2020-12"";""DE03200907002535050071"";""EDEKDEHHXXX"";""-78","14"";"""";"""";""58016075002246221218114732""; ""27.12.2018"";""27.12.2018"";""Kartenzahlung/-abrechnung"";""Kaufland Potsdam//Potsdam/DE / KAUFLAND"";""2018-12-24T09:07 Debitk.1 2020-12"";""DE25600501010004001050"";""SOLADEST600"";""-4","38"";"""";"""";""61327685068888241218090734""; ""27.12.2018"";""27.12.2018"";""Lastschrift"";""PayPal Europe S.a.r.l. et Cie S.C.A"";""PP.9674.PP . PLAYSTATION","Ihr Einkauf bei PLAYSTATION AWV-MELDEPFLICHT BEACHTEN HOTLINE BUNDESBANK: (0800) 1234-111"";""FR7630004021180001013711792"";""BNPAFRPPXXX"";""-17","99"";""LU96ZZZ0000000000000000058 "";""4EEJ224PC6BAL "";""1004569810505 PP.9674.PP PAYPAL""; ""27.12.2018"";""27.12.2018"";""Kartenzahlung/-abrechnung"";""DM FIL.2740 H:65128//POTSDAM/DE / DM DROGERIEMARKT SAGT DANKE"";""2018-12-22T12:06 Debitk.1 2020-12"";""DE88600501017402099997"";""SOLADEST600"";""-11","10"";"""";"""";""65128268003696221218120652""; ""27.12.2018"";""27.12.2018"";""Lastschrift"";""PayPal Europe S.a.r.l. et Cie S.C.A"";""PP.9674.PP . GREYZONE","Ihr Einkauf bei GREYZONE AWV-MELDEPFLICHT BEACHTEN HOTLINE BUNDESBANK: (0800) 1234-111"";""FR7630004021180001013711792"";""BNPAFRPPXXX"";""-115","00"";""LU96ZZZ0000000000000000058 "";""4EEJ224PC6BAL "";""1004567744595 PP.9674.PP PAYPAL""; ""27.12.2018"";""27.12.2018"";""Kartenzahlung/-abrechnung"";""EDEKA 2372//POTSDAM/DE / EDEKA 2372"";""2018-12-22T10:49 Debitk.1 2020-12"";""DE03200907002535050071"";""EDEKDEHHXXX"";""-6","56"";"""";"""";""58019489000132221218104948""; ""27.12.2018"";""27.12.2018"";""Kartenzahlung/-abrechnung"";""Aral Tankstelle Potsdam Grossbeerenstrase 121//Potsdam/DE / ARAL AG"";""Kartenzahlung 2018-12-25T19:39 Debitk.1 2020-12"";""DE95430800830802029200"";""DRESDEFF430"";""-8","97"";"""";"""";""71015812499393251218193900"";"


In [26]:
encoding = "ISO-8859-1"
pd.read_csv("CommonSign_DEV.csv",sep=";")

,Buchungstag,Wertstellung,Buchungstext,Auftraggeber / Beguenstigter,Verwendungszweck,Kontonummer,BLZ,Betrag (EUR),Glaeubiger-ID,Mandatsreferenz,Kundenreferenz,Unnamed: 11
0,22.01.2019,22.01.2019,Lastschrift,PayPal Europe S.a.r.l. et Cie S.C.A,"PP.9674.PP . LIEFERHELD, Ihr Einkauf bei LIEFE...",FR7630004021180001013711792,BNPAFRPPXXX,"-12,40",LU96ZZZ0000000000000000058,4EEJ224PC6BAL,NaN,NaN
1,21.01.2019,21.01.2019,Geldautomat,Potsdam//Potsdam/DE / SPARDA-BANK BERLIN,2019-01-18T19:03 Debitk.1 2020-12 Fremdentgelt...,DE31120965972004061500,GENODEF1S10,"-154,90",NaN,NaN,00001056025337180119190321,NaN
2,21.01.2019,21.01.2019,Kartenzahlung/-abrechnung,EDEKA 2372//POTSDAM/DE / EDEKA 2372,2019-01-19T11:42 Debitk.1 2020-12,DE03200907002535050071,EDEKDEHHXXX,"-39,67",NaN,NaN,58016060008607190119114212,NaN
3,18.01.2019,18.01.2019,Kartenzahlung/-abrechnung,EDEKA 2372//POTSDAM/DE / EDEKA 2372,2019-01-17T19:56 Debitk.1 2020-12,DE03200907002535050071,EDEKDEHHXXX,"-9,76",NaN,NaN,58016060008131170119195651,NaN
4,17.01.2019,17.01.2019,Kartenzahlung/-abrechnung,Burger King Store 25460 Frankfurt S//FRANKFURT...,2019-01-15T17:36 Debitk.1 2020-12,DE03500800000096743900,DRESDEFFXXX,"-9,14",NaN,NaN,52080888001245150119173604,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
74,27.12.2018,27.12.2018,Lastschrift,PayPal Europe S.a.r.l. et Cie S.C.A,"PP.9674.PP . PLAYSTATION, Ihr Einkauf bei PLAY...",FR7630004021180001013711792,BNPAFRPPXXX,"-17,99",LU96ZZZ0000000000000000058,4EEJ224PC6BAL,1004569810505 PP.9674.PP PAYPAL,NaN
75,27.12.2018,27.12.2018,Kartenzahlung/-abrechnung,DM FIL.2740 H:65128//POTSDAM/DE / DM DROGERIEM...,2018-12-22T12:06 Debitk.1 2020-12,DE88600501017402099997,SOLADEST600,"-11,10",NaN,NaN,65128268003696221218120652,NaN
76,27.12.2018,27.12.2018,Lastschrift,PayPal Europe S.a.r.l. et Cie S.C.A,"PP.9674.PP . GREYZONE, Ihr Einkauf bei GREYZON...",FR7630004021180001013711792,BNPAFRPPXXX,"-115,00",LU96ZZZ0000000000000000058,4EEJ224PC6BAL,1004567744595 PP.9674.PP PAYPAL,NaN
77,27.12.2018,27.12.2018,Kartenzahlung/-abrechnung,EDEKA 2372//POTSDAM/DE / EDEKA 2372,2018-12-22T10:49 Debitk.1 2020-12,DE03200907002535050071,EDEKDEHHXXX,"-6,56",NaN,NaN,58019489000132221218104948,NaN


In [ ]:
pd.read_csv("CommonSign_DEV.csv",sep=";")